In [4]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from models import imnet_resnet

import importlib
importlib.reload(imnet_resnet)


<module 'models.imnet_resnet' from '/juice/scr/ananya/cifar_experiments/custom_training/models/imnet_resnet.py'>

In [6]:
model = imnet_resnet.ResNet50(pretrained=True, pretrain_style='mocov2', checkpoint_path='/u/scr/ananya/simclr_weights/moco_v2_800ep_pretrain.pth.tar')

In [7]:
model.fc

ModuleAttributeError: 'ResNet50' object has no attribute 'fc'